In [1]:
import torch

## 1. Simplified Attention

Say we want to compute self-attention output for 2nd token `x^2` in position, which is `journey`

### 1.1. Sample Toy Input

Word embedding are made to be just 3D dimensional

In [2]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

### 1.2. Steps

#### 1.2.1. Computing Attention Scores

Dot Product operation is used mathematically represent how two vectors are aligned
> Dot Product of query `x^2` with every other token in sequence

In [6]:
query = inputs[1]

attn_scores_2 = torch.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(query, x_i)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


#### 1.2.2. Normalise Attention Scores

* Main Goal behind the normalization to obtain attention weights which can sum to 1

* It makes it use full for convention and interpretation

We use softmax instead of summation function as it offers

* Positive Values -> Probability
* Favorable Gradient Properties

##### 1.2.2.1. Summation Normalisation [NOT USED]

In [7]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


##### 1.2.2.2. Softmax Normalisation

In [9]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)
print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())


## using pytorch inbuilt softmax function
attn_weights_2 = torch.nn.functional.softmax(attn_scores_2, dim=0)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())



Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)
Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


#### 1.2.3. Updated Context Vector

This final step, updates initial context vector of `x^2` using attention weights derived in previous steps

In [10]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])
